In [ ]:
# ============================
# Robustness & Fairness
# Baseline / Typos / Noise / Drift / Bias-Mitigation
# Dependencies: numpy, pandas, scikit-learn, matplotlib
# ============================

import os, random, string, math
import numpy as np
import pandas as pd
from collections import defaultdict

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    roc_curve
)

import matplotlib.pyplot as plt

# ----------------------------
# 1) Утилиты
# ----------------------------

def set_seed(seed: int):
    random.seed(seed); np.random.seed(seed)

def make_synthetic_dataset(n=5000, n_num=10, p_group=0.5, group_bias=0.6, seed=42):
    """
    Генерируем табличный датасет:
    - numeric features: X_num ~ N(0,1)
    - categorical feature 'state' из {A..F}
    - protected attribute 'group' in {0,1}
    - y зависит от линейной комбинации + смещение по группе (group_bias)
    """
    set_seed(seed)
    # числовые признаки
    X_num = np.random.randn(n, n_num)
    # категориальный признак "state"
    states = np.array(list("ABCDEF"))
    state = np.random.choice(states, size=n, replace=True)
    # защищённый признак
    group = (np.random.rand(n) < p_group).astype(int)

    # генерим веса и логит
    w = np.random.randn(n_num)
    lin = X_num @ w
    # добавим слабую зависимость от категории (state)
    state_effect = {s: np.random.uniform(-0.5, 0.5) for s in states}
    lin += np.vectorize(lambda s: state_effect[s])(state)

    # добавим смещение по группе (создаём потенциальный биас)
    lin += (group * group_bias)

    # вероятность класса 1
    prob = 1 / (1 + np.exp(-lin))
    y = (np.random.rand(n) < prob).astype(int)

    df = pd.DataFrame(X_num, columns=[f"f{i}" for i in range(n_num)])
    df["state"] = state
    df["group"] = group
    df["y"] = y
    return df

def inject_typos(series, typo_rate=0.05, seed=0):
    """Вносит опечатки в строковые значения: заменяет один символ в части значений."""
    set_seed(seed)
    def corrupt(s):
        if s is None or len(s) == 0: return s
        idx = np.random.randint(0, len(s))
        letters = string.ascii_uppercase
        new_char = random.choice(letters.replace(s[idx], "")) if s[idx] in letters else random.choice(letters)
        return s[:idx] + new_char + s[idx+1:]
    mask = np.random.rand(len(series)) < typo_rate
    out = series.astype(str).copy()
    out[mask] = out[mask].apply(corrupt)
    return out

def add_numeric_noise(X, sigma=0.5, seed=0):
    set_seed(seed)
    return X + np.random.normal(0, sigma, size=X.shape)

def demographic_parity_gap(y_pred, group):
    """|P(yhat=1|A=1) - P(yhat=1|A=0)|"""
    g1 = y_pred[group==1].mean() if (group==1).any() else 0.0
    g0 = y_pred[group==0].mean() if (group==0).any() else 0.0
    return float(abs(g1 - g0))

def tpr(y_true, y_pred):
    tp = ((y_true==1) & (y_pred==1)).sum()
    p  = (y_true==1).sum()
    return 0.0 if p==0 else tp / p

def equal_opportunity_gap(y_true, y_pred, group):
    """|TPR(A=1) - TPR(A=0)|"""
    tpr1 = tpr(y_true[group==1], y_pred[group==1]) if (group==1).any() else 0.0
    tpr0 = tpr(y_true[group==0], y_pred[group==0]) if (group==0).any() else 0.0
    return float(abs(tpr1 - tpr0))

def metrics_all(y_true, y_pred, y_prob, group):
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "macro_precision": precision_score(y_true, y_pred, average="macro", zero_division=0),
        "macro_recall": recall_score(y_true, y_pred, average="macro", zero_division=0),
        "macro_f1": f1_score(y_true, y_pred, average="macro", zero_division=0),
        "roc_auc": roc_auc_score(y_true, y_prob) if len(np.unique(y_true))==2 else np.nan,
        "DPG": demographic_parity_gap(y_pred, group),
        "EOG": equal_opportunity_gap(y_true, y_pred, group)
    }

def group_threshold_postprocessing(y_prob, y_true, group, grid=np.linspace(0.2, 0.8, 31)):
    """
    Простая bias-mitigation: подбираем два порога (для group=0 и group=1),
    минимизируя EOG; при равенстве — максимизируем accuracy.
    """
    best = None
    for t0 in grid:
        for t1 in grid:
            thr = np.where(group==1, t1, t0)
            y_hat = (y_prob >= thr).astype(int)
            eog = equal_opportunity_gap(y_true, y_hat, group)
            acc = accuracy_score(y_true, y_hat)
            score = ( -eog, acc )  # меньше EOG, больше accuracy
            if (best is None) or (score > best[0]):
                best = (score, (t0, t1), y_hat)
    (_, _), (t0, t1), y_hat = best
    return (t0, t1), y_hat

# ----------------------------
# 2) Конфиг эксперимента
# ----------------------------

N_RUNS = 5
TEST_SIZE = 0.3
SEED_BASE = 123

os.makedirs("figures", exist_ok=True)

# Колонки
NUM_COLS = [f"f{i}" for i in range(10)]
CAT_COLS = ["state"]
ALL_INPUT = NUM_COLS + CAT_COLS

# пайплайн препроцессинга + модель
preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), NUM_COLS),
        ("cat", OneHotEncoder(handle_unknown="ignore"), CAT_COLS),
    ]
)
model = LogisticRegression(max_iter=2000)
pipe = Pipeline(steps=[("prep", preprocess), ("clf", model)])

# ----------------------------
# 3) Основной цикл
# ----------------------------

records = []

for run in range(N_RUNS):
    seed = SEED_BASE + run
    df = make_synthetic_dataset(
        n=5000, n_num=10, p_group=0.5, group_bias=0.6, seed=seed
    )

    X = df[ALL_INPUT + ["group"]].copy()  # group не кодируем, но храним отдельно
    y = df["y"].values
    g = df["group"].values

    # train/test split (стратифицируем по y)
    X_train, X_test, y_train, y_test, g_train, g_test = train_test_split(
        X[ALL_INPUT], y, g, test_size=TEST_SIZE, stratify=y, random_state=seed
    )

    # ---- Baseline ----
    pipe.fit(X_train, y_train)
    prob_base = pipe.predict_proba(X_test)[:,1]
    pred_base = (prob_base >= 0.5).astype(int)
    m_base = metrics_all(y_test, pred_base, prob_base, g_test)
    records.append({"run": run, "scenario": "Baseline", **m_base})

    # ---- Typos (опечатки в категориальном) ----
    X_test_typos = X_test.copy()
    X_test_typos["state"] = inject_typos(X_test_typos["state"], typo_rate=0.05, seed=seed)
    prob_typos = pipe.predict_proba(X_test_typos)[:,1]
    pred_typos = (prob_typos >= 0.5).astype(int)
    m_typos = metrics_all(y_test, pred_typos, prob_typos, g_test)
    records.append({"run": run, "scenario": "Typos", **m_typos})

    # ---- Noise (шум в числовых) ----
    X_test_noise = X_test.copy()
    X_test_noise[NUM_COLS] = add_numeric_noise(X_test_noise[NUM_COLS].values, sigma=0.5, seed=seed)
    prob_noise = pipe.predict_proba(X_test_noise)[:,1]
    pred_noise = (prob_noise >= 0.5).astype(int)
    m_noise = metrics_all(y_test, pred_noise, prob_noise, g_test)
    records.append({"run": run, "scenario": "Noise", **m_noise})

    # ---- Drift (10% заменяем другими наблюдениями с перекошенной группой) ----
    # формируем "дрейфовую" подвыборку: больше объектов группы 1
    df_drift = make_synthetic_dataset(n=1000, n_num=10, p_group=0.8, group_bias=0.6, seed=seed+777)
    X_drift = df_drift[ALL_INPUT]
    y_drift = df_drift["y"].values
    g_drift = df_drift["group"].values

    X_test_drift = X_test.copy()
    y_test_drift = y_test.copy()
    g_test_drift = g_test.copy()

    k = max(1, int(0.10 * len(X_test_drift)))
    idx_replace = np.random.choice(len(X_test_drift), size=k, replace=False)
    idx_take = np.random.choice(len(X_drift), size=k, replace=False)

    X_test_drift.iloc[idx_replace] = X_drift.iloc[idx_take].values
    y_test_drift[idx_replace] = y_drift[idx_take]
    g_test_drift[idx_replace] = g_drift[idx_take]

    prob_drift = pipe.predict_proba(X_test_drift)[:,1]
    pred_drift = (prob_drift >= 0.5).astype(int)
    m_drift = metrics_all(y_test_drift, pred_drift, prob_drift, g_test_drift)
    records.append({"run": run, "scenario": "Drift", **m_drift})

    # ---- Bias-Mitigation (подбор порогов по группам для снижения EOG) ----
    (t0, t1), pred_mitig = group_threshold_postprocessing(prob_base, y_test, g_test)
    m_mitig = metrics_all(y_test, pred_mitig, prob_base, g_test)  # y_prob тот же; пороги иные
    records.append({"run": run, "scenario": "Bias-Mitigation", **m_mitig})

# ----------------------------
# 4) Сохранение результатов
# ----------------------------

df_runs = pd.DataFrame.from_records(records)
df_runs.to_csv("per_run_results.csv", index=False)

df_mean = (
    df_runs.groupby("scenario")
    .agg({
        "accuracy":"mean","macro_precision":"mean","macro_recall":"mean",
        "macro_f1":"mean","roc_auc":"mean","DPG":"mean","EOG":"mean"
    })
    .reset_index()
)
df_mean.to_csv("mean_results.csv", index=False)

# Совместим для «metrics_summary.csv» как в пакете
df_mean.round(3).to_csv("metrics_summary.csv", index=False)

print("Saved: per_run_results.csv, mean_results.csv, metrics_summary.csv")

# ----------------------------
# 5) Графики
# ----------------------------

# Accuracy bar
plt.figure(figsize=(7,4))
plt.bar(df_mean["scenario"], df_mean["accuracy"])
plt.title("Accuracy by scenario")
plt.ylabel("Accuracy")
plt.xticks(rotation=20)
plt.tight_layout()
plt.savefig("figures/accuracy_bar.png", dpi=150)
plt.close()

# Robustness (как линия по Accuracy)
plt.figure(figsize=(7,4))
order = ["Baseline","Typos","Noise","Drift","Bias-Mitigation"]
acc_vals = [df_mean.set_index("scenario").loc[s,"accuracy"] for s in order]
plt.plot(order, acc_vals, marker="o")
plt.title("Robustness (Accuracy across scenarios)")
plt.ylabel("Accuracy")
plt.xticks(rotation=20)
plt.tight_layout()
plt.savefig("figures/robustness_plot.png", dpi=150)
plt.close()

# Fairness plot: DPG и EOG
plt.figure(figsize=(7,4))
dpg_vals = [df_mean.set_index("scenario").loc[s,"DPG"] for s in order]
eog_vals = [df_mean.set_index("scenario").loc[s,"EOG"] for s in order]
plt.plot(order, dpg_vals, marker="o", label="DPG")
plt.plot(order, eog_vals, marker="o", label="EOG")
plt.title("Fairness gaps (DPG, EOG)")
plt.ylabel("Gap")
plt.xticks(rotation=20)
plt.legend()
plt.tight_layout()
plt.savefig("figures/fairness_plot.png", dpi=150)
plt.close()

# ROC для baseline
# Берем последний прогон (run=N_RUNS-1), baseline
last_run = df_runs[df_runs["run"]==N_RUNS-1]
# Чтобы получить prob для ROC, пересчитаем на последнем ране (быстро)
seed = SEED_BASE + (N_RUNS-1)
df = make_synthetic_dataset(n=5000, n_num=10, p_group=0.5, group_bias=0.6, seed=seed)
X = df[ALL_INPUT]; y = df["y"].values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, stratify=y, random_state=seed
)
pipe.fit(X_train, y_train)
prob_base = pipe.predict_proba(X_test)[:,1]
fpr, tpr_vals, _ = roc_curve(y_test, prob_base)

plt.figure(figsize=(5,5))
plt.plot(fpr, tpr_vals, label=f"ROC (AUC={roc_auc_score(y_test, prob_base):.3f})")
plt.plot([0,1],[0,1], linestyle="--")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC curve (Baseline)")
plt.legend()
plt.tight_layout()
plt.savefig("figures/roc_curve.png", dpi=150)
plt.close()

print("Saved figures: figures/accuracy_bar.png, figures/robustness_plot.png, figures/fairness_plot.png, figures/roc_curve.png")

# ----------------------------
# 6) LaTeX-таблица
# ----------------------------

def to_latex_table(df):
    cols_order = ["scenario","accuracy","macro_precision","macro_recall","macro_f1","roc_auc","DPG","EOG"]
    df2 = df[cols_order].copy()
    df2.columns = ["Scenario","Accuracy","Macro P","Macro R","Macro F1","ROC-AUC","DPG↓","EOG↓"]
    return df2.round(3).to_latex(index=False, escape=False, caption="Mean metrics across scenarios", label="tab:metrics")

latex = to_latex_table(df_mean)
with open("table.tex","w", encoding="utf-8") as f:
    f.write(latex)

print("Saved LaTeX table: table.tex")

# ----------------------------
# 7) Короткий вывод чисел на экран
# ----------------------------

print("\n=== Mean results ===")
display(df_mean.round(3))
